In [ ]:
#modèle ResNet val_acc inconnu mais prometteur - à tester - dataset images détourées
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout, GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, Callback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
from timeit import default_timer as timer
import numpy as np
import os
import const

# Définition des chemins
diverted_images_train_path = os.path.join(const.DATASET_WO_BACKGROUND, "train")
diverted_images_valid_path = os.path.join(const.DATASET_WO_BACKGROUND, "valid")
diverted_images_test_path = os.path.join(const.DATASET_WO_BACKGROUND, "test")

# Définition de la taille du lot
batch_size = 16

# Définition des callbacks
reduce_learning_rate = ReduceLROnPlateau(monitor="val_loss", patience=2, min_delta=0.01, factor=0.1, cooldown=4, verbose=1)
early_stopping = EarlyStopping(patience=3, min_delta=0.01, verbose=1, mode='min', monitor='val_loss')

class TimingCallback(Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

time_callback = TimingCallback()

# Création des générateurs d'images
train_generator = ImageDataGenerator().flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size)
valid_generator = ImageDataGenerator().flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size)
test_generator = ImageDataGenerator().flow_from_directory(diverted_images_test_path, target_size=(224, 224), batch_size=batch_size)

# Récupération du nombre de classes
num_classes = train_generator.num_classes

# Construction du modèle
def build_model():
    base_model = ResNet50(weights='imagenet', include_top=False)
    for layer in base_model.layers: 
        layer.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(rate=0.2)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(rate=0.2)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

model = build_model()

# Compilation du modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

# Entraînement du modèle
history_diverted = model.fit(train_generator,
                    epochs=15,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

# Évaluation du modèle
test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])
